# Q1. Downloading the data
We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".
Download the data for January and February 2023.
Read the data for January. How many columns are there?

In [54]:
!python -V
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

Python 3.11.9


In [55]:
df = pd.read_parquet('./data/yellow_tripdata_2023-01.parquet')
df.shape

(3066766, 19)

# Q2. Computing duration
Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

In [56]:
df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

In [57]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
df['duration'].std()  

42.59435124195458

# Q3. Dropping outliers
Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

In [58]:
df_2 = df[(df.duration >= 1) & (df.duration <= 60)]

In [59]:
len(df_2)/len(df)*100

98.1220282212598

# Q4. One-hot encoding
Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
Fit a dictionary vectorizer
Get a feature matrix from it
What's the dimensionality of this matrix (number of columns)?

In [60]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [61]:
df_train = read_dataframe('./data/yellow_tripdata_2023-01.parquet')
df_val = read_dataframe('./data/yellow_tripdata_2023-02.parquet')

In [62]:
len(df_train), len(df_val)

(3009173, 2855951)

In [63]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

categorical = ['PU_DO']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [64]:
train_dicts

[{'PU_DO': '161_141', 'trip_distance': 0.97},
 {'PU_DO': '43_237', 'trip_distance': 1.1},
 {'PU_DO': '48_238', 'trip_distance': 2.51},
 {'PU_DO': '138_7', 'trip_distance': 1.9},
 {'PU_DO': '107_79', 'trip_distance': 1.43},
 {'PU_DO': '161_137', 'trip_distance': 1.84},
 {'PU_DO': '239_143', 'trip_distance': 1.66},
 {'PU_DO': '142_200', 'trip_distance': 11.7},
 {'PU_DO': '164_236', 'trip_distance': 2.95},
 {'PU_DO': '141_107', 'trip_distance': 3.01},
 {'PU_DO': '234_68', 'trip_distance': 1.8},
 {'PU_DO': '79_264', 'trip_distance': 7.3},
 {'PU_DO': '164_143', 'trip_distance': 3.23},
 {'PU_DO': '138_33', 'trip_distance': 11.43},
 {'PU_DO': '33_61', 'trip_distance': 2.95},
 {'PU_DO': '79_186', 'trip_distance': 1.52},
 {'PU_DO': '90_48', 'trip_distance': 2.23},
 {'PU_DO': '113_255', 'trip_distance': 4.5},
 {'PU_DO': '237_239', 'trip_distance': 1.2},
 {'PU_DO': '143_229', 'trip_distance': 2.5},
 {'PU_DO': '137_79', 'trip_distance': 1.4},
 {'PU_DO': '132_116', 'trip_distance': 17.8},
 {'PU_DO'

# Q5. Training a model
Now let's use the feature matrix from the previous step to train a model.

Train a plain linear regression model with default parameters
Calculate the RMSE of the model on the training data
What's the RMSE on train?

In [65]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

C:\Users\wongs\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


5.144453437201816

# Q6. Evaluating the model
Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

In [66]:
y_pred = lr.predict(X_val)
mean_squared_error(y_val, y_pred, squared=False)

C:\Users\wongs\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


5.256177130486534